# Live Plotting
(back to overview [offline](../Main.ipynb),[online](https://nbviewer.jupyter.org/github/QCoDeS/Qcodes/tree/master/docs/examples/Main.ipynb))


[read on nbviewer](https://nbviewer.jupyter.org/github/QCoDeS/Qcodes/tree/master/docs/examples/plotting/live_plotting.ipynb)

## TL;DR
Enable live plotting by specifying 'subscripition.default_subscriber' in your qcodesrc.json.

## Introduction
Live plotting in qcodes can be achieved by sending the acquired data to other applications. For this an adapter for the specific application is necessary. These adapter register as subscribers to the qcodes dataset and expose a callback that gets called on regular intervals that can be specified.

## Live plotting with plottr as an example
An example of a live plotting application is for example [*plottr*](https://github.com/kouwenhovenlab/plottr). After installing *plottr* you can use it by simply setting `subscription.default_subscribers` to `["Plottr"]` in your `qcodesrc.json` file or alternatively by updating your qcodes config for your current session:

In [12]:
import qcodes
qcodes.config.subscription.default_subscribers = ["Plottr"]

With this simple chang you can run all your notebooks without any changes and live data will be streamed to plottr. For that lets consider the simple example adapted from the dataset docs:

In [2]:
%matplotlib notebook
import numpy as np
from time import sleep
from qcodes import find_or_create_instrument, initialise_database, new_experiment, Measurement
from qcodes.tests.instrument_mocks import DummyInstrument

In [3]:
dac = find_or_create_instrument(DummyInstrument, 'dac', gates=['ch1'], recreate=True)
dmm = find_or_create_instrument(DummyInstrument, 'dmm', gates=['v1'], recreate=True)

def customgetter():
    return 5*np.exp(-0.2*dac.ch1()) + 0.02*5*np.random.randn()
    
dmm.v1.get = customgetter

In [4]:
initialise_database()
new_experiment(name='tutorial_exp', sample_name="no sample")

tutorial_exp#no sample#7@./exp_container_tutorial.db
----------------------------------------------------

Running the next cell should render the data in the live plotting front end:

In [11]:
meas = Measurement()
meas.register_parameter(dac.ch1)  # register the first independent parameter
meas.register_parameter(dmm.v1, setpoints=(dac.ch1,))  # now register the dependent oone
# make live plotting faster:
meas.write_period = 0.1

with meas.run() as datasaver:      
    for set_v in np.linspace(0, 25, 100):
        dac.ch1.set(set_v)
        get_v = dmm.v1.get()
        datasaver.add_result((dac.ch1, set_v),
                             (dmm.v1, get_v))
        # we added some extra sleep so that the live plotting can be observed
        sleep(0.1)

Starting experimental run with id: 17
